In [ ]:
import numpy as np
import pandas as pd

import gc
from catboost import CatBoostClassifier

In [ ]:
df = pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv').append(pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv')).reset_index(drop=True)
df

In [ ]:
df['min'] = df[df['target'].notnull()].min(axis=1)
df['max'] = df[df['target'].notnull()].max(axis=1)
df['std'] = df[df['target'].notnull()].std(axis=1)

In [ ]:
train_df = df[df['target'].notnull()]
test_df = df[df['target'].isnull()]
del df
gc.collect()

In [ ]:
y = train_df["target"]
y = y.astype(float)
X = train_df.drop(["id", "target"], axis=1)

In [ ]:
params = {'iterations': 1000,
          'learning_rate': 0.09153154807802073,
          'depth': 7,
          'objective': 'CrossEntropy',
          'colsample_bylevel': 0.0829041193408479,
          'boosting_type': 'Plain',
          'bootstrap_type': 'MVS'}

model = CatBoostClassifier(random_state=1, verbose=0, **params).fit(X, y)

In [ ]:
sub_x = test_df.drop(["id", "target"], axis=1)
y_prob_sub = model.predict_proba(sub_x)[:, 1]
test_df['target'] = y_prob_sub
test_df[['id', 'target']].to_csv('submission.csv', index=False)